# LangChain と PaLM API で PDF ドキュメントの要約を行う例

LangChain のパッケージ、および、PDF の扱いに必要なパッケージをインストールします。

In [ ]:
!pip install --user \
  langchain==0.0.294 transformers==4.34.0 \
  pypdf==3.16.1 cryptography==41.0.4

NISC が一般公開している「[インターネットの安全・安心ハンドブック](https://security-portal.nisc.go.jp/guidance/handbook.html)」のプロローグ部分をダウンロードします。





In [1]:
!wget -q https://raw.githubusercontent.com/enakai00/cloud_genAI_app_book/main/PDF/handbook-prologue.pdf

ダウンロードした PDF を読み込んで、全体を1つのテキストにまとめます。

In [2]:
from langchain.document_loaders import PyPDFLoader

pages = PyPDFLoader('handbook-prologue.pdf').load()
document = ''
for page in pages:
    document += page.page_content

PaLM API を利用するクライアントオブジェクトを取得します。

In [3]:
from langchain.llms import VertexAI
llm = VertexAI(model_name='text-bison@002',
               temperature=0.1, max_output_tokens=1024)

RecursiveCharacterTextSplitter, load_qa_chain, AnalyzeDocumentChain モジュールの使用例です。

`input_document` に与えたテキストに基づいて、`question` の質問に回答します。

In [49]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=6000, chunk_overlap=200)
qa_chain = load_qa_chain(llm, chain_type='map_reduce')
qa_document_chain = AnalyzeDocumentChain(
    combine_docs_chain=qa_chain, text_splitter=text_splitter)

qa_document_chain.run(
    input_document='今は６月で雨が多い時期です。', question='最近の天候は？')

' 最近の天候は雨が多いです。'

`input_document` に与えたテキストの内容をまとめるように指示して結果を得る関数を用意します。

In [42]:
def get_description(document):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000, chunk_overlap=200)
    qa_chain = load_qa_chain(llm, chain_type='map_reduce')
    qa_document_chain = AnalyzeDocumentChain(
        combine_docs_chain=qa_chain, text_splitter=text_splitter)

    prompt = '何についての文書ですか？日本語で200字程度にまとめて教えてください。'

    description = qa_document_chain.run(
        input_document=document, question=prompt)
    return description

先に用意した PDF ファイルを適用します。

In [43]:
print(get_description(document))

 この文書は、サイバー攻撃について説明しています。サイバー攻撃とは、コンピュータやネットワークを介して行われる攻撃のことです。サイバー攻撃には、マルウェアの感染、フィッシング詐欺、ランサムウェア、ボットネット、BEC（ビジネスメール詐欺）などがあります。これらの攻撃は、個人や企業に大きな被害をもたらす可能性があります。


任意の質問に答える関数を用意します。

In [37]:
def get_answer(document, question):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000, chunk_overlap=200)
    qa_chain = load_qa_chain(llm, chain_type='refine')
    qa_document_chain = AnalyzeDocumentChain(
        combine_docs_chain=qa_chain, text_splitter=text_splitter)

    prompt = '{} 日本語で200字程度にまとめて教えてください。'.format(question)
    answer = qa_document_chain.run(
        input_document=document, question=prompt)
    return answer

先に用意した PDF ファイルに基づいて、質問の回答を取得します。

In [41]:
question = 'サイバーセキュリティ対策のポイントを箇条書きにまとめてください。'
print(get_answer(document, question))

 サイバーセキュリティ対策のポイント

・OSやソフトウェアは常に最新の状態にしておく。
・多要素認証を利用する。
・偽メールや偽サイトに騙されないように用心する。
・スマホやPCの画面ロックを利用する。
・大切な情報は失う前にバックアップする。
・外出先では紛失・盗難・覗き見に注意する。
・困ったときは1人で悩まず、まず相談する。
・パスワードは長く複雑にして、他と使い回さないようにする。
・心当たりのない送信元からのメールに添付されているファイルやリンクには注意する。
